In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [2]:
import ipywidgets as widgets
from IPython.display import display
import plotly.express as pxc
from ipywidgets import interact


In [3]:
import pandas as pd

# Laden der Datei in ein Pandas DataFrame
datei_path = r"C:\Users\Kunde\Desktop\BNT_Technikerschule\Dr. Movsessian\Projekt Windkraftanlage\Features_for_SHAP32rpm_repaierd.txt"
df = pd.read_csv(datei_path)

# Anzeigen der ersten Zeilen des DataFrames
df.tail()


# ...

,Cov1_1,Cov1_2,Cov1_4,Cov1_5,Cov1_7,Cov1_8,Cov1_10,Cov1_11,Cov2_2,Cov2_4,...,DC2_RPM,Wind_Direction,Temperature,Preciptation,Generator_on_off,Power_kW,Pitch_deg,Yaw,Azimuth,novelty_index
2922,1926.096405,22.077982,175.857296,-45.826991,50.419635,-196.514162,-19.894280,17.230319,106.262023,73.319511,...,32.025182,225.3840,2.175221,0.005802,1.0,-0.203927,-0.204647,10.02373,-35.567223,19.300004
2923,1968.053957,1.849336,237.957362,-40.600944,37.026843,-172.434916,-17.566236,12.396130,92.356970,53.739386,...,32.024936,225.4025,2.175308,0.005802,1.0,-0.203776,-0.166080,10.02100,-35.075370,18.866672
2924,1878.377866,17.217453,313.894413,-33.844109,56.978583,-157.683324,-14.607295,7.955903,96.110466,57.960094,...,32.025416,224.6777,2.154097,0.005802,1.0,-0.202933,-0.446031,10.02232,-27.235961,19.676828
2925,1883.592661,56.197118,452.159911,-29.978124,108.109518,-139.086048,6.094632,11.027323,106.358605,72.326554,...,32.026613,220.8360,2.157872,0.005802,1.0,-0.202147,-0.475663,10.02020,6.454405,19.960807
2926,1888.385538,48.195217,431.133481,-29.426011,87.764201,-129.770940,21.244808,12.426216,108.380538,70.841707,...,32.031763,213.0807,2.180657,0.005802,1.0,-0.202552,-0.527526,10.03005,104.349985,22.522516


# Anzeige in Tabs (Jupyter Widgets)
## Mit Jupyter Widgets können Sie die Spalten in Tabs aufteilen, um die Anzeige zu erleichter

In [4]:
def display_df_in_tabs(df, chunk_size=10):
    num_chunks = (len(df.columns) + chunk_size - 1) // chunk_size
    for i in range(num_chunks):
        start = i * chunk_size
        end = min((i + 1) * chunk_size, len(df.columns))
        chunk = df.iloc[:, start:end]
        tab = widgets.Tab(children=[widgets.Output(layout=widgets.Layout(width='auto')) for _ in range(len(chunk.columns))],
                          titles=list(chunk.columns))
        for j, col in enumerate(chunk.columns):
            with tab.children[j]:
                display(chunk[col])
        display(tab)

display_df_in_tabs(df)



In [5]:
df_2=df
print(df_2["Generator_on_off"].value_counts())


Generator_on_off
1.000000    2127
0.000000      31
0.888542       1
0.807813       1
0.701042       1
0.854167       1
0.658333       1
0.752083       1
0.475521       1
0.744271       1
0.857812       1
0.846354       1
Name: count, dtype: int64


In [6]:
df.columns

Index(['Cov1_1', 'Cov1_2', 'Cov1_4', 'Cov1_5', 'Cov1_7', 'Cov1_8', 'Cov1_10',
       'Cov1_11', 'Cov2_2', 'Cov2_4', 'Cov2_5', 'Cov2_7', 'Cov2_8', 'Cov2_10',
       'Cov2_11', 'Cov4_4', 'Cov4_5', 'Cov4_7', 'Cov4_8', 'Cov4_10', 'Cov4_11',
       'Cov5_5', 'Cov5_7', 'Cov5_8', 'Cov5_10', 'Cov5_11', 'Cov7_7', 'Cov7_8',
       'Cov7_10', 'Cov7_11', 'Cov8_8', 'Cov8_10', 'Cov8_11', 'Cov10_10',
       'Cov10_11', 'Cov11_11', 'WindSpeed', 'DC1_RPM', 'DC2_RPM',
       'Wind_Direction', 'Temperature', 'Preciptation', 'Generator_on_off',
       'Power_kW', 'Pitch_deg', 'Yaw', 'Azimuth', 'novelty_index'],
      dtype='object')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2927 entries, 0 to 2926
Data columns (total 48 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Cov1_1            2927 non-null   float64
 1   Cov1_2            2927 non-null   float64
 2   Cov1_4            2927 non-null   float64
 3   Cov1_5            2927 non-null   float64
 4   Cov1_7            2927 non-null   float64
 5   Cov1_8            2927 non-null   float64
 6   Cov1_10           2927 non-null   float64
 7   Cov1_11           2927 non-null   float64
 8   Cov2_2            2927 non-null   float64
 9   Cov2_4            2927 non-null   float64
 10  Cov2_5            2927 non-null   float64
 11  Cov2_7            2927 non-null   float64
 12  Cov2_8            2927 non-null   float64
 13  Cov2_10           2927 non-null   float64
 14  Cov2_11           2927 non-null   float64
 15  Cov4_4            2927 non-null   float64
 16  Cov4_5            2927 non-null   float64


In [8]:

df.describe()

,Cov1_1,Cov1_2,Cov1_4,Cov1_5,Cov1_7,Cov1_8,Cov1_10,Cov1_11,Cov2_2,Cov2_4,...,DC2_RPM,Wind_Direction,Temperature,Preciptation,Generator_on_off,Power_kW,Pitch_deg,Yaw,Azimuth,novelty_index
count,2927.000000,2927.000000,2927.000000,2927.000000,2927.000000,2927.000000,2927.000000,2927.000000,2927.000000,2927.000000,...,2927.000000,2927.000000,2927.000000,2927.000000,2168.000000,2168.000000,2927.000000,2168.000000,2927.000000,2927.000000
mean,1747.931863,-2.260068,317.196271,-28.444858,46.515793,-159.859453,10.256604,18.023993,95.290987,65.304500,...,31.982444,219.890276,2.618653,0.009267,0.984588,-0.146232,-0.365317,9.947981,10.156601,6.983072
std,199.708307,39.504559,151.510188,19.806876,69.737599,27.411937,20.140408,9.595882,12.554723,30.339935,...,0.251218,80.914737,2.770315,0.071789,0.120005,0.431149,0.340793,0.661750,102.590188,3.508187
min,989.239237,-145.223071,-78.720347,-96.682045,-140.913708,-304.416136,-65.375506,-14.446204,65.348420,7.278890,...,30.000287,0.131618,-3.371019,-0.000454,0.000000,-0.880806,-1.426433,4.000000,-179.463434,2.764926
25%,1624.058779,-32.330924,189.466139,-41.199322,-10.279779,-177.269372,-5.901096,12.420366,85.870861,42.781594,...,32.009364,191.184350,0.959786,0.000000,1.000000,-0.214719,-0.484065,10.007517,-74.649448,4.726142
50%,1750.020173,-5.972418,321.332371,-28.991283,49.155240,-161.358001,10.987156,18.572368,94.012226,59.270103,...,32.025193,229.806000,2.269189,0.005649,1.000000,-0.197195,-0.263347,10.017120,13.881518,6.049983
75%,1876.425325,29.254731,442.783770,-16.136016,108.141518,-143.200508,25.728132,24.983788,103.875741,82.512411,...,32.030999,278.209200,4.478815,0.005802,1.000000,-0.175742,-0.124634,10.041622,100.276608,7.826569
max,2924.941275,106.863926,733.283649,51.777710,189.242296,-54.493735,73.917371,41.616833,145.757134,183.928483,...,32.102752,359.859600,11.945510,1.000000,1.000000,7.898488,-0.000035,10.136890,179.890022,31.580461


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

# Dummy-Daten erstellen (falls nötig)
# df = pd.read_csv('dein_windkraftdaten.csv')

# Output-Widget erstellen
output = widgets.Output()

def erstelle_pairplot(szenario):
    with output:  # Nutzt das Output-Widget, um doppelte Ausgaben zu verhindern
        clear_output(wait=True)  # Vorherige Plots entfernen
        
        if szenario == 'Wind- und Leistungsdaten':
            spalten = ['WindSpeed', 'Power_kW', 'DC1_RPM', 'DC2_RPM']
            titel = 'Pairplot: Wind- und Leistungsdaten'
        elif szenario == 'Umweltbedingungen und Leistung':
            spalten = ['Temperature', 'Preciptation', 'Power_kW']
            titel = 'Pairplot: Umweltbedingungen und Leistung'
        elif szenario == 'Ausrichtung und Leistung':
            spalten = ['Pitch_deg', 'Yaw', 'Azimuth', 'Power_kW']
            titel = 'Pairplot: Ausrichtung und Leistung'
        elif szenario == 'Kovarianz-Daten':
            spalten = ['Cov1_1', 'Cov2_2', 'Cov4_4', 'Cov5_5', 'Cov7_7', 'Cov8_8', 'Cov10_10', 'Cov11_11']
            titel = 'Pairplot: Kovarianz-Daten'
        else:
            print("Ungültiges Szenario")
            return

        try:
            plot = sns.pairplot(df[spalten])  # Pairplot erstellen
            plot.fig.suptitle(titel, y=1.02)  # Titel setzen
            plt.show()
        except Exception as e:
            print(f"Fehler: {e}")

# Dropdown für die Szenario-Auswahl
dropdown = widgets.Dropdown(options=['Wind- und Leistungsdaten', 'Umweltbedingungen und Leistung', 'Ausrichtung und Leistung', 'Kovarianz-Daten'], description='Szenario:')

# Verknüpfe Dropdown mit der Funktion
dropdown.observe(lambda change: erstelle_pairplot(change['new']), names='value')

# Anzeige
display(dropdown, output)

# Initiale Anzeige
erstelle_pairplot(dropdown.value)


Dropdown(description='Szenario:', options=('Wind- und Leistungsdaten', 'Umweltbedingungen und Leistung', 'Ausr…

Output()

In [14]:




relevante_spalten = [
    "WindSpeed", "Wind_Direction", "Temperature", "Preciptation",  # Umweltbedingungen
    "DC1_RPM", "DC2_RPM", "Power_kW", "Pitch_deg", "Yaw", "Azimuth",  # Betriebsdaten
    "Generator_on_off", # Status 
]

# Falls "Cov1"-"Cov36" Sensordaten sind und du sie behalten willst:
cov_spalten = [col for col in df.columns if col.startswith("Cov")]

# Endgültige Spaltenliste
features = relevante_spalten + cov_spalten

# ✅ . Daten filtern
df = df[features]

# ✅ . Fehlende Werte behandeln (hier: Entfernen von Zeilen mit NaN)
df = df.dropna()

# ✅ . Skalierung der numerischen Werte (außer Generator_on_off, da 0/1)
scaler = MinMaxScaler()
numerische_spalten = [col for col in df.columns if col != "Generator_on_off"]
df[numerische_spalten] = scaler.fit_transform(df[numerische_spalten])

# ✅ 6. Ergebnis anzeigen
print(df.head())

# Optional: Daten speichern
# df.to_csv("vorbereitete_daten.csv", index=False)


   WindSpeed  Wind_Direction  Temperature  Preciptation   DC1_RPM   DC2_RPM  \
0   0.145542        0.838095     0.363770      0.953488  0.968860  0.946192   
1   0.112612        0.863321     0.359408      0.953488  0.970191  0.949021   
2   0.105434        0.851611     0.358454      0.953488  0.950558  0.930401   
3   0.104907        0.877472     0.355140      0.953488  0.914142  0.895160   
4   0.093385        0.853076     0.350759      0.953488  0.205302  0.779602   

   Power_kW  Pitch_deg       Yaw   Azimuth  ...    Cov7_7    Cov7_8   Cov7_10  \
0  0.078971   0.431931  0.980381  0.720479  ...  0.646088  0.486132  0.488080   
1  0.079062   0.489718  0.975601  0.969982  ...  0.615289  0.304033  0.458650   
2  0.079076   0.122621  0.976113  0.551947  ...  0.713952  0.634155  0.647971   
3  0.079094   0.048987  0.980301  0.400981  ...  0.602873  0.511900  0.432717   
4  0.078988   0.047906  0.983583  0.381388  ...  0.594061  0.424264  0.377281   

    Cov7_11    Cov8_8   Cov8_10   Cov8

In [15]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

# ✅ Zeilenindex als Ersatz für Zeitstempel
df["Index"] = df.index  

# ✅ Funktion zur Erstellung der Diagramme
def plot_schadensindex(typ):
    plt.figure(figsize=(8, 5))

    if typ == "Trend (Index)":
        plt.plot(df["Index"], df["Schadensindex"], color="red", label="Schadensindex")
        plt.xlabel("Datenindex")
        plt.ylabel("Schadensindex")
        plt.title("Schadensindex über die Datenreihenfolge")
        plt.legend()

    elif typ == "Windgeschwindigkeit":
        plt.scatter(df["WindSpeed"], df["Schadensindex"], alpha=0.5, color="blue")
        plt.xlabel("Windgeschwindigkeit (m/s)")
        plt.ylabel("Schadensindex")
        plt.title("Zusammenhang zwischen Windgeschwindigkeit und Schadensindex")

    elif typ == "Temperatur":
        plt.scatter(df["Temperature"], df["Schadensindex"], alpha=0.5, color="green")
        plt.xlabel("Temperatur (°C)")
        plt.ylabel("Schadensindex")
        plt.title("Einfluss der Temperatur auf den Schadensindex")

    plt.grid()
    plt.show()

# ✅ Dropdown-Widget für Auswahl
dropdown = widgets.Dropdown(
    options=["Trend (Index)", "Windgeschwindigkeit", "Temperatur"],
    value="Trend (Index)",
    description="Diagramm:",
)

# ✅ Anzeige aktualisieren, wenn Auswahl geändert wird
widgets.interactive(plot_schadensindex, typ=dropdown)


interactive(children=(Dropdown(description='Diagramm:', options=('Trend (Index)', 'Windgeschwindigkeit', 'Temp…

<Figure size 800x500 with 0 Axes>